# HSLS GWAS Template: Continuous, Quantitative Outcome

- This template goes through the necessary joining, cleaning, and tests to run a simplified GWAS with a continuous, quantitative outcome. (preset to look at chromosome 1). 
- Throughout the template, there are small sections that will require input from you, the user. 
- **Please go through the template and fill out these required sections before running the cells.** 
- **They will be prefaced with the following comment:**

`####################################### PLEASE EDIT THIS SECTION #######################################`

## Import Packages/Paths

- This section is for importing the `hail`, `pandas`, and `os` packages.
- It is also saving the Google storage paths to the Workspace Bucket (`bucket`) and the WGS Exome HailMatrix Table (`exome_split_mt_path`).

In [ ]:
import hail as hl
import os
import pandas as pd
from hail.plot import show
from bokeh.plotting import output_file, save
import bokeh.io
from bokeh.io import *
from bokeh.resources import INLINE
bokeh.io.output_notebook(INLINE) 
%matplotlib inline

In [ ]:
# save the workspace bucket path
bucket = os.getenv('WORKSPACE_BUCKET')

# this saves the path as a variable
exome_split_mt_path = os.getenv("WGS_EXOME_SPLIT_HAIL_PATH")

In [ ]:
# SET HG38 AS DEFAULT REF.
hl.init(default_reference="GRCh38")

# Import Phenotype Data

- Below, the template is **importing** your saved .tsv file as a **Hail Table**. 
- It must be a tab-separated file.

In [ ]:
####################################### PLEASE EDIT THIS SECTION #######################################
file_name = "file_name_here.tsv"

In [ ]:
# copy tsv file from the bucket to the current working space
os.system(f"gsutil cp '{bucket}/data/{file_name}' .")
pheno = f'{bucket}/data/{file_name}'
pheno_df = (hl.import_table(pheno,
                              types={'person_id':hl.tstr},
                              impute=True,
                              key='person_id'))

# Import Genotype Data

- <em>AllofUs</em> provides short variants for all the 245,394 short read WGS participants that are stored in the format of a [Hail VDS](https://support.researchallofus.org/hc/en-us/articles/14927774297620-The-new-VariantDataset-VDS-format-for-All-of-Us-short-read-WGS-data).

- This template uses the **Exome subset with split rows for alleles**.

In [ ]:
# read the path in as a HailMatrix Table
exome_split_mt = hl.read_matrix_table(exome_split_mt_path)

# check the dimensions of the HailMatrix Table
exome_split_mt.count()

## Pulling Specific Regions of the Genome

- **Filter for genome region you interested in**. It is preset to chromosome 1. 
- Note: If you would like to run a full GWAS, comment this section out. Be careful, though, it is costly and time consuming to run a full GWAS.

In [ ]:
####################################### PLEASE EDIT THIS SECTION [OPTIONAL] #######################################
# this is the chromosome interval to extract this variant
test_intervals = ['chr1']

# the function to filter variant region
exome_split_mt = hl.filter_intervals(
    exome_split_mt,
    [hl.parse_locus_interval(x,)
     for x in test_intervals])

## Merging Genotype and Phenotype Data

We must **merge** all of the genotype and phenotype data into a **final HailMatrix Table**.

In [ ]:
# this merges the phenotype data with the genotype data
cohort_mt = exome_split_mt.semi_join_cols(pheno_df)
cohort_mt = cohort_mt.annotate_cols(pheno = pheno_df[cohort_mt.s])

# Auxiliary Tables

- <em>AllofUs</em> provides tables with additional information on those in the cohort, depending on what type of genomic data you are using.
- We will be using two of these tables which give **ancestry** and **relatedness** information on those in our cohort.
- There is [more information on auxiliary tables](https://support.researchallofus.org/hc/en-us/articles/4614687617556-How-the-All-of-Us-Genomic-data-are-organized) provided by AoU.

In [ ]:
# store the path string that loads auxiliary tables
auxiliary_path = "gs://fc-aou-datasets-controlled/v7/wgs/short_read/snpindel/aux"

## Ancestry Table

In [ ]:
# get the ancestry path inside auxiliary path
ancestry_path = f'{auxiliary_path}/ancestry'

# save the ancestry .tsv file path as a variable
ancestry_pred_path = f'{ancestry_path}/ancestry_preds.tsv'

# import as a Hail Table
ancestry_pred = hl.import_table(ancestry_pred_path,
                               key="research_id", 
                               impute=True, 
                               types={"research_id":"tstr","pca_features":hl.tarray(hl.tfloat)})

# annotates each person id with their ancestry information
cohort_mt = cohort_mt.annotate_cols(anc = ancestry_pred[cohort_mt.s])

## Relatedness Table

- Import a list of **related samples** to remove from `cohort_mt`.

In [ ]:
# get the relatedness path inside auxiliary path
relatedness = f'{auxiliary_path}/relatedness'

# save the relatedness 
related_samples_path = f'{relatedness}/relatedness_flagged_samples.tsv'

# import as Hail Table
related_remove = hl.import_table(related_samples_path,
                                 types={"sample_id":"tstr"},
                                key="sample_id")

# removes sample ids that are related (kinship score > 0.1)
final_mt = cohort_mt.anti_join_cols(related_remove)

In [ ]:
# check difference in sample size
# print(cohort_mt.count())
# print(final_mt.count())

# Stratification

- Stratify the linear regression by choosing your stratification variable `strat_variable` and the group you are interested in within the stratification variable `strat_type`. 

- An example would be `strat_variable = final_mt.pheno.sex_at_birth` and `strat_type = "Male"`.

In [ ]:
####################################### PLEASE EDIT THIS SECTION [OPTIONAL] #######################################
strat_variable = insert_strat_variable
strat_type = "insert_strat_type"

**You must uncomment the cell below (delete the `#`) for the stratification to run!**

In [ ]:
#final_mt = final_mt.filter_cols(strat_variable == strat_type)

# Statistical Tests

## Quality Checks

Here we will check:
- **minor allele frequency (MAF)**
- **Hardy–Weinberg equilibrium (HWE)**

As a note: The genomic data is preprocessed for **sex discrepancy** and **heterozygosity rate (AB >= 0.2)** by AoU.

### MAF

- MAF thresholds should be sample size-dependent; typically 0.01 for large (N=100,000) and 0.05 for moderate (N=10,000) samples.

In [ ]:
####################################### PLEASE EDIT THIS SECTION [OPTIONAL] #######################################
MAF_threshold = 0.01

In [ ]:
final_mt = hl.variant_qc(final_mt)
final_mt = final_mt.filter_rows(hl.min(final_mt.variant_qc.AF) > MAF_threshold, keep = True)
#final_mt.count()

### Hardy-Weinberg Equilibrium

- In this section, we are making sure that our variants meet Hardy-Weinberg equilibrium. 
- It is preset to a p-value of `1e-6` to check genotyping quality, but can be adjusted depending on sample size and type of analysis.
- I am using the HWE threshold suggested [in this paper](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6001694/).

In [ ]:
####################################### PLEASE EDIT THIS SECTION [OPTIONAL] #######################################
hwe_threshold = 1e-6

In [ ]:
final_mt = final_mt.filter_rows(final_mt.variant_qc.p_value_hwe > hwe_threshold, keep = True)
#final_mt.count()

## Linear Regression

Here, we will run a **linear regression**:
- a **numerical outcome** of your choice
- number of alternate alleles as the **predictor** (Additive Model)
- **covariates** adjusted for in the regression

In [ ]:
# look at variables in HailMatrix Table
final_mt.describe()

In [ ]:
####################################### PLEASE EDIT THIS SECTION #######################################
outcome = insert_outcome_here
covar = [insert_covariates_here] # separate items with commas ","

### Unadjusted

Running the regression **without adjustment** for covariates:

In [ ]:
unadj_reg = hl.linear_regression_rows(
    y = outcome,
    x = final_mt.GT.n_alt_alleles(),
    covariates = [1.0]
)

unadj_reg.show()

### Adjusted

Running the regression **with adjustment** for covariates:

In [ ]:
adj_reg = hl.linear_regression_rows(
    y = outcome,
    x = final_mt.GT.n_alt_alleles(),
    covariates = [1.0] + covar
)

adj_reg.show()

### Saving Results
The code below will save your adjusted regression results. The current file name is `adj_reg`. You can adjust the file name by using the `file_name` variable.


In [ ]:
file_name = "adj_reg"

In [ ]:
adj_reg_flat = adj_reg.flatten()

adj_reg_save_path = f'{bucket}/data/{file_name}.tsv.bgz'

adj_reg_flat.export(adj_reg_save_path)

Uncomment the code below to load adjusted regression results back into the notebook.

In [ ]:
# adj_reg_results = hl.import_table(adj_reg_save_path, 
#                                   types={"locus": hl.tstr, "alleles": hl.tarray(hl.tstr), 
#                                          "beta": hl.tfloat64, "p_value": hl.tfloat64, 
#                                          "fit.n_iterations": hl.tint32})

# adj_reg_results = adj_reg_results.annotate(locus=hl.parse_locus(adj_reg.locus, reference_genome='GRCh38'))

### Manhattan Plot - Adjusted Regression

- This section creates a manhatten plot of the adjusted regression results. It then saves the plot to your workspace under `"manhattan.html"`.
- Uncomment `show_bokeh(p)` to view the plot.

In [ ]:
def show_bokeh(plot_fig):
    try:
        bokeh.plotting.reset_output()
        bokeh.plotting.output_notebook()
        bokeh.plotting.show(plot_fig)
    except:
        bokeh.plotting.output_notebook()
        bokeh.plotting.show(plot_fig)

In [ ]:
p = hl.plot.manhattan(adj_reg.p_value)
#show_bokeh(p)

In [ ]:
output_file("manhattan.html")
save(p)

In [ ]:
#copy saved bokeh plot to the workspace bucket.
!gsutil cp manhattan.html {bucket}/data/